# QSOF - Task 2 - Bonus Question
This was the original problem statement:

> Implement a circuit that returns |01> and |10> with equal probability.
>
> Requirements :
- The circuit should consist only of CNOTs, RXs and RYs. 
- Start from all parameters in parametric gates being equal to 0 or randomly chosen. 
- You should find the right set of parameters using gradient descent (you can use more advanced optimization methods if you like). 
- Simulations must be done with sampling - i.e. a limited number of measurements per iteration and noise. 
>
> Compare the results for different numbers of measurements: 1, 10, 100, 1000. 
>
> **Bonus question:**
>
> **How to make sure you produce state |01> + |10> and not |01> - |10> ?**

I tackled the first part of the problem in [another notebook](QOSF%20-%20Task%202%20%28with%20QisKit%29.ipynb) using a variational method. 

In this notebook I want to use the bonus question to understand the theory behind Variational Quantum Eigensolvers (VQE) in more detail. Also this time I've used the [Julia programming language](https://julialang.org/) which I've been learning as an alternative to Python and offers great support for numerial computing research.

## Some theory

For a given state $\psi$, quantum mechanics tells us that the expectation value $E(\psi)$ of an operator $H$ is a scalar measure of the projection of the state onto the operator's basis: 
$E(\psi) =\langle \psi|H|\psi \rangle$ 

So to generate a desired state with high probability with respect to H, we would want to _maximise_ the projection of the state $\psi$ onto H. In other words, to find a state for which the expectation of finding it when measured in H's basis is as close to 1 as possible:
$\langle \psi|H|\psi \rangle \approx 1$

For this to be true, it's _sufficient_ that: 
$H=|\psi\rangle \langle\psi|$

Which can be seen by substitution:
$\langle \psi|H|\psi \rangle = \langle \psi|\psi\rangle \langle\psi|\psi \rangle = 1$

----

In our case, the eigenstate we're looking for is: 
$\psi = \frac{1}{\sqrt{2}}(|01\rangle + |10\rangle)$

An therefore the operator would be: 
$
H=\frac{1}{2}\begin{bmatrix}
0 & 0 & 0 & 0 \\
0 & 1 & 1 & 0 \\
0 & 1 & 1 & 0 \\
0 & 0 & 0 & 0
\end{bmatrix}
$

This line of reasoning is easy to check with [Wolfram Alpha](https://www.wolframalpha.com/input/?i=eigenvectors+0.5*%7B%7B0%2C0%2C0%2C0%7D%2C%7B0%2C1%2C1%2C0%7D%2C%7B0%2C1%2C1%2C0%7D%2C%7B0%2C0%2C0%2C0%7D%7D) or numerically in Julia:

In [1]:
# Uncomment the first time you run this notebook
#]add StatsBase LinearAlgebra Yao Evolutionary

using LinearAlgebra

ψ = sqrt(0.5)*[0, 1 + 0im, 1 + 0im, 0]
H = ψ*ψ'

eigen(H)

Eigen{Complex{Float64},Float64,Array{Complex{Float64},2},Array{Float64,1}}
values:
4-element Array{Float64,1}:
 0.0
 0.0
 5.551115123125783e-16
 1.0000000000000002
vectors:
4×4 Array{Complex{Float64},2}:
 1.0+0.0im  0.0+0.0im        0.0+0.0im       0.0+0.0im
 0.0+0.0im  0.0+0.0im   0.707107+0.0im  0.707107-0.0im
 0.0+0.0im  0.0+0.0im  -0.707107-0.0im  0.707107+0.0im
 0.0+0.0im  1.0+0.0im        0.0+0.0im       0.0+0.0im

The eigenvalues of the matrix are ${0,0,0,1}$ with the only non-zero eigenvalue corresponding to the state we're looking for. So the problem is to generate, with an ansatz, an approximation to the eigenstate which corresponds to the largest eigenvalue of this operator, or equivalently, to minimise the negative of the eigenvalue.

Now, if we can find a way to optimise the parameters of our ansatz based on this expectation value instead of using the probability distribution distance metric then we will be approximating the exact target state rather than just the target distribution. Therefore we'll be sure to produce $|01\rangle + |10\rangle$ rather than $|01\rangle - |10\rangle$!

Let's try it out in practice. I'm going to use the [Yao.jl](https://yaoquantum.org/) which is a nice quantum simulator written for Julia. First we prepare the environment and some simple convenience methods:

In [2]:
using Yao
using StatsBase: mean

# A function to round displayed results so they0re easier to understand
rounded(x) = round(x;digits=3);

## The ansatz

We can define the ansatz as [before](QOSF%20-%20Task%202%20(with%20QisKit).ipynb#1.-Designing-and-testing-the-ansatz). The parameters are given by the array $\theta$:

In [3]:
ansatz(θ) = chain(2, 
                put(1 => Rx(θ[1])),
                put(2 => Rx(θ[2])), 
                put(1 => Ry(θ[3])),
                put(2 => Ry(θ[4])),
                control(1, 2 => X))

ansatz([π/2,π/2,π/2,π/2])

nqubits: 2
chain
├─ put on (1)
│  └─ rot(X, 1.5707963267948966)
├─ put on (2)
│  └─ rot(X, 1.5707963267948966)
├─ put on (1)
│  └─ rot(Y, 1.5707963267948966)
├─ put on (2)
│  └─ rot(Y, 1.5707963267948966)
└─ control(1)
   └─ (2,) X


## The objective function

To perform the optimisation the Yao simulator conveniently allows us to measure in any basis we like so the objective function directly measures the result of the ansatz in the H basis using the `matblock` function to convert our Hamiltonian to a unitary operator. This is cheating a bit but we'll come back to how to do this on a real device using the computational basis a bit later.

In [4]:
measure_H_basis(nshots) = r -> measure(matblock(H), r; nshots=nshots)

# Generate the state and measure in H basis then take the negative
objective_function(θ; nshots=100) = 
    zero_state(2) |> ansatz(θ) |> measure_H_basis(nshots) |> mean |> -

objective_function (generic function with 1 method)

# Training

In the previous notebook we used gradient based optimisation to train the circuit. This time, just for a change, we'll use an evolutionary algorithm to do the same. Julia has an excellent package called [Evolutionary.jl](https://wildart.github.io/Evolutionary.jl/stable/) which is very easy to use. We just import the package, pass in the objective function and the initial parameters and let it go. We tell it to use [CMA-ES](https://wildart.github.io/Evolutionary.jl/stable/cmaes/) as the evolution strategy which seems to work quite nicely.

In [5]:
using Evolutionary

res = Evolutionary.optimize(
            objective_function, 
            rand(4)*2π, 
            CMAES(),
            Evolutionary.Options(iterations=200, reltol=0.001))


 * Status: success

 * Candidate solution
    Minimizer:  [6.350566447785466, 6.277935892236361, 1.5829698698036074,  ...]
    Minimum:    -0.9900000000000001
    Iterations: 62

 * Found with
    Algorithm: (15,30)-CMA-ES


In [6]:
θopt = Evolutionary.minimizer(res)
ψ = zero_state(2) |> ansatz(θopt) |> statevec .|> rounded

4-element Array{Complex{Float64},1}:
 0.024 - 0.001im
 0.711 - 0.024im
 0.702 + 0.024im
 0.024 - 0.003im

It gives us an approximation to the state we were asked for including a random global phase which we could factor out if we wanted.

In [7]:
normalize(ψ/ψ[2]) .|> rounded

4-element Array{Complex{Float64},1}:
 0.024 - 0.0im
 0.711 + 0.0im
 0.701 + 0.048im
 0.024 - 0.002im

## Measuring in the computational basis

So the theory seems to work but we still have a problem because most quantum devices only allow measurement in the computational basis. This means that somehow we have to rotate the results from the Hamiltonian basis back to the computational basis to be able to perform the measurement. It turns out that we can reduce H to a sequence of [Pauli matrix operations](https://en.wikipedia.org/wiki/Pauli_matrices):

$$4H = I\otimes I+X\otimes X+Y\otimes Y-Z\otimes Z$$

Since the expectation is linear we can evaluate it for each term individually and then add them at the end. It looks like this:
$$4\langle \psi|H|\psi \rangle = \langle \psi|I\otimes I|\psi \rangle + \langle \psi|X\otimes X|\psi \rangle + \langle \psi|Y\otimes Y|\psi \rangle - \langle \psi|Z\otimes Z|\psi \rangle$$

Or, more concisely as:
$$4\langle H \rangle = \langle I\otimes I \rangle + \langle X\otimes X \rangle + \langle Y\otimes Y \rangle - \langle Z\otimes Z \rangle$$

We can then use the following identity and apply it to each term in turn:
$$
\langle \psi_1\otimes \psi_2 |U_1\otimes U_2|\psi_1\otimes \psi_2 \rangle = 
\langle \psi_1 |U_1|\psi_1 \rangle
\langle \psi_2 |U_2|\psi_2 \rangle
$$

It's immediately clear from this identity that the expectation of $I$ is simply the constant 1 (the inner product of two vectors with themselves). It also tells us that the expectation of the XX, YY and ZZ terms is the product of the expectations of the individual measurements.

The XX and YY terms also need [additional rotations](https://www.mustythoughts.com/variational-quantum-eigensolver-explained) to align them with the computational basis for measurement. The ZZ term needs no rotation because it's already in the right basis but notice the `-` sign which is a factor to be applied to the final expectation sum. 

Finally, we want to find the expectation with respect to the Z operator eigenstates rather than the computational basis measurements. To do that we must transform the measurements $\{0,1\}$ to the Z-basis eigenvalues $\{1,-1\}$, respectively, such that $0 \Rightarrow 1$ and $1 \Rightarrow -1$.

In [8]:
XX = repeat(2, Ry(-π/2))
YY = repeat(2, Rx(π/2))
ZZ = repeat(2, I2)

# Convert a measurement {true,false} to Z-operator basis {-1,1}
to_Z_basis(c) = c==0 ? 1 : -1

# Measure a state multiple times and convert 
measure_Z_basis(ψ) = measure(ψ; nshots=100) .|> c -> to_Z_basis(c[1])*to_Z_basis(c[2])

# Calculate the expectation for a particular term, applying appropriate rotation before measurement
expectation_for_term(term_rotation) =
    θ -> zero_state(2) |> ansatz(θ) |> term_rotation |> measure_Z_basis |> mean |> -

expectation(θ) = 
    1 + expectation_for_term(XX)(θ) + expectation_for_term(YY)(θ) - expectation_for_term(ZZ)(θ)

res = Evolutionary.optimize(
        expectation,
        rand(4)*2π, 
        CMAES(),
        Evolutionary.Options(iterations=200, reltol=0.001))


 * Status: success

 * Candidate solution
    Minimizer:  [3.119950655610562, 3.101673288820813, 4.718870260541731,  ...]
    Minimum:    -1.98
    Iterations: 97

 * Found with
    Algorithm: (15,30)-CMA-ES


To check the result we run the ansatz with the optimised parameters and normalise it to remove the global phase.

In [9]:
θopt = Evolutionary.minimizer(res)
ψ = zero_state(2) |> ansatz(θopt) |> statevec
normalize(ψ/ψ[2]) .|> rounded

4-element Array{Complex{Float64},1}:
 -0.003 + 0.014im
  0.709 - 0.0im
  0.705 + 0.015im
 -0.003 + 0.014im

This is a very good approximation to the H based measurement. We can even run these parameters through the original objective function and get a perfect agreement.

In [10]:
objective_function(θopt)

-1.0000000000000002

# Conclusions

This notebook takes a closer look at the maths behind the VQE solution to the original problem. We used transformations of the measurement basis to estimate the expectation value with respect to the target state and used it as the objective function for training.

With this more rigourous approach we can differentiate between states which would otherwise result in the same measurement probability distribution.

In this notebook we used an evolutionary optimiser instead of a gradient based one and it performed very well. It was usually able to get good results within 100 or so iterations without ans specific tuning. Like the previous notebook, it'd be interesting to systematically compare the performance of this optimisation schemes.

This optimisation is not done in the presence of noise. I couldn't find a simple way to add a noise model as can be done in QisKit (maybe something that could be done as a separate project?)


# References

**Variational Quantum Eigensolver explained** - Musty Thoughts blog
https://www.mustythoughts.com/variational-quantum-eigensolver-explained

**Variational Quantum Eigensolver** (Excellent worked example) - Davit Khachatryan
https://nbviewer.jupyter.org/github/DavitKhach/quantum-algorithms-tutorials/blob/master/variational_quantum_eigensolver.ipynb

**Yao.jl** - A General Purpose Quantum Computation Simulation Framework
https://docs.yaoquantum.org/stable/

**Evolutionary.jl** Julia evolutionary algorithms package https://wildart.github.io/Evolutionary.jl/stable/

**Basic Quantum Mechanics** - Klaus Ziock (1969)